In [ ]:
import numpy as np


In [ ]:
class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stopping conditions
        if len(unique_classes) == 1:
            return {'class': unique_classes[0]}

        if self.max_depth is not None and depth >= self.max_depth:
            return {'class': np.bincount(y).argmax()}

        best_gain = -1
        best_split = None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])
            for threshold in thresholds:
                left_mask = X[:, feature_idx] <= threshold
                right_mask = ~left_mask

                if len(y[left_mask]) == 0 or len(y[right_mask]) == 0:
                    continue

                gain = self._information_gain(y, y[left_mask], y[right_mask])

                if gain > best_gain:
                    best_gain = gain
                    best_split = (feature_idx, threshold, left_mask, right_mask)

        if best_split is None:
            return {'class': np.bincount(y).argmax()}

        feature_idx, threshold, left_mask, right_mask = best_split

        return {
            'feature_idx': feature_idx,
            'threshold': threshold,
            'left': self._build_tree(X[left_mask], y[left_mask], depth + 1),
            'right': self._build_tree(X[right_mask], y[right_mask], depth + 1)
        }

    def _entropy(self, y):
        probs = np.bincount(y) / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))

    def _information_gain(self, parent, left, right):
        return self._entropy(parent) - (
            len(left)/len(parent) * self._entropy(left) +
            len(right)/len(parent) * self._entropy(right)
        )

    def predict(self, X):
        return np.array([self._predict_sample(x, self.tree) for x in X])

    def _predict_sample(self, x, node):
        if 'class' in node:
            return node['class']
        if x[node['feature_idx']] <= node['threshold']:
            return self._predict_sample(x, node['left'])
        else:
            return self._predict_sample(x, node['right'])


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [ ]:
data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Step 3: Train & Evaluate Custom Decision Tree

In [ ]:
custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

y_pred_custom = custom_tree.predict(X_test)
accuracy_custom = accuracy_score(y_test, y_pred_custom)

print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")


Custom Decision Tree Accuracy: 1.0000


Step 4: Scikit-Learn Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

sk_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sk_tree.fit(X_train, y_train)

y_pred_sklearn = sk_tree.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")


Scikit-learn Decision Tree Accuracy: 1.0000


Step 5: Accuracy Comparison

In [ ]:
print("Accuracy Comparison")
print("Custom Decision Tree:", accuracy_custom)
print("Scikit-learn Decision Tree:", accuracy_sklearn)


Accuracy Comparison
Custom Decision Tree: 1.0
Scikit-learn Decision Tree: 1.0


PART 2: Ensemble Methods & Hyperparameter Tuning (Wine Dataset) Step 1: Load Wine Dataset

In [ ]:
from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score


In [ ]:
X, y = load_wine(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Step 2: Decision Tree vs Random Forest (Classification)

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)

dt.fit(X_train, y_train)
rf.fit(X_train, y_train)

dt_f1 = f1_score(y_test, dt.predict(X_test), average='weighted')
rf_f1 = f1_score(y_test, rf.predict(X_test), average='weighted')

print("Decision Tree F1:", dt_f1)
print("Random Forest F1:", rf_f1)


Decision Tree F1: 0.9439974457215836
Random Forest F1: 1.0


Step 3: Hyperparameter Tuning (Random Forest – Classification)

In [ ]:
from sklearn.model_selection import GridSearchCV


In [ ]:
param_grid = {
    'n_estimators': [50, 100],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5]
}

grid = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=5,
    scoring='f1_weighted'
)

grid.fit(X_train, y_train)

print("Best Parameters:", grid.best_params_)


Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 100}


In [ ]:
best_rf = grid.best_estimator_
best_f1 = f1_score(y_test, best_rf.predict(X_test), average='weighted')

print("Optimized Random Forest F1:", best_f1)


Optimized Random Forest F1: 1.0


PART 3: Regression – Decision Tree & Random Forest

In [ ]:
from sklearn.datasets import load_diabetes
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV


In [ ]:
X, y = load_diabetes(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Step 2: Train Regression Models

In [ ]:
dt_reg = DecisionTreeRegressor(random_state=42)
rf_reg = RandomForestRegressor(random_state=42)

dt_reg.fit(X_train, y_train)
rf_reg.fit(X_train, y_train)

print("DT MSE:", mean_squared_error(y_test, dt_reg.predict(X_test)))
print("RF MSE:", mean_squared_error(y_test, rf_reg.predict(X_test)))


Step 3: Hyperparameter Tuning (RandomizedSearchCV)

In [ ]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10]
}

rand_search = RandomizedSearchCV(
    RandomForestRegressor(random_state=42),
    param_dist,
    n_iter=10,
    cv=5,
    scoring='neg_mean_squared_error'
)

rand_search.fit(X_train, y_train)

best_rf_reg = rand_search.best_estimator_

print("Best Parameters:", rand_search.best_params_)
print("Optimized RF MSE:", mean_squared_error(y_test, best_rf_reg.predict(X_test)))


Best Parameters: {'n_estimators': 200, 'min_samples_split': 5, 'max_depth': 5}
Optimized RF MSE: 2874.1730568192197


Final Conclusion :

Custom Decision Tree helps understand entropy & information gain

Scikit-learn trees are optimized and more accurate

Random Forest improves performance by reducing variance

Hyperparameter tuning significantly improves generalization

GridSearch is exhaustive; RandomizedSearch is efficient for large spaces